![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

#### Setting up environment

In [1]:
// We need to load assemblies at the start in their own cell
#load "../Initialize.csx"

In [2]:
// QuantBook C# Research Environment
// For more information see https://www.quantconnect.com/docs/v2/our-platform/research/getting-started
#load "../QuantConnect.csx"


#### Loading up crypto data

In [3]:
// Import namespaces
using QuantConnect;
using QuantConnect.Data.Market;
using QuantConnect.Data;
using QuantConnect.Brokerages;
using System;
using System.Linq;
using System.Collections.Generic;
using QuantConnect.Research;

// Initialize QuantBook
var qb = new QuantConnect.Research.QuantBook();

qb.SetAccountCurrency("USDT");
qb.SetBrokerageModel(BrokerageName.Binance, AccountType.Cash);

var btcUsdt = qb.AddCrypto("BTCUSDT", Resolution.Hour, Market.Binance).Symbol;

// Date range
var startDate = new DateTime(2013, 04, 07);
var endDate = new DateTime(2024, 11, 1);

var btcHistoryHour = qb.History<TradeBar>(btcUsdt, startDate, endDate, Resolution.Hour).ToList();
Console.WriteLine($"Number of hourly data points: {btcHistoryHour.Count}");
foreach (var bar in btcHistoryHour.Take(5))
{
    Console.WriteLine($"Date: {bar.EndTime}, Open: {bar.Open}, High: {bar.High}, Low: {bar.Low}, Close: {bar.Close}, Volume: {bar.Volume}");
}

#### Calcul du rendement du Benchmark (HODL)

Pour évaluer les performances de notre stratégie horaire, nous devons comparer ses résultats avec un benchmark simple, ici un hodl sur BTC. 

Le benchmark suppose un investissement initial de **5000 USDT** converti en BTC à la première heure du backtest. Nous suivons ensuite l'évolution de cet investissement en fonction des prix horaires de clôture jusqu'à la fin de la période étudiée.

Ce calcul fournit une référence pour analyser si la stratégie ajoutée surperforme ou sous-performe par rapport à un investissement passif en BTC.


In [ ]:
//// Assuming initialInvestment remains the same (5000 USDT):
decimal initialInvestment = 5000m;

// We already have btcHistoryHour as a List<TradeBar>
// We'll compute a cumulative return as if we bought BTC at the first hour's close and held throughout.

// Get the first close price
decimal firstClose = btcHistoryHour.First().Close;
decimal runningTotal = initialInvestment;

var cumulativeReturnHour = new List<(DateTime Time, decimal PortfolioValue)>();

foreach (var bar in btcHistoryHour)
{
    runningTotal = initialInvestment * (bar.Close / firstClose);
    cumulativeReturnHour.Add((bar.EndTime, runningTotal));
}

// Display some returns
Console.WriteLine("Benchmark (HODL) portfolio evolution:");
foreach (var point in cumulativeReturnHour.Take(5))
{
    Console.WriteLine($"DateTime: {point.Time}, Portfolio Value: {point.PortfolioValue:C}");
}

Console.WriteLine($"Final Portfolio Value: {cumulativeReturnHour.Last().Time}: {cumulativeReturnHour.Last().PortfolioValue:C} BTC Close {btcHistoryHour.Last().Close}$/BTC");


#### Aggegating hours to daily

In [4]:
var dailyData = btcHistoryHour
    .GroupBy(x => x.EndTime.Date)
    .Select(g => new TradeBar
    {
        Time = g.Key,
        Open = g.First().Open,
        High = g.Max(x => x.High),
        Low = g.Min(x => x.Low),
        Close = g.Last().Close,
        Volume = g.Sum(x => x.Volume),
        Symbol = btcUsdt
    })
    .OrderBy(x => x.Time)
    .ToList();


// Maintenant dailyData contient une liste de TradeBars quotidiens reconstitués depuis l'horaire.


// Afficher quelques lignes pour vérifier
foreach (var bar in dailyData.Take(5))
{
    Console.WriteLine($"{bar.Time.ToShortDateString()} O:{bar.Open} H:{bar.High} L:{bar.Low} C:{bar.Close}");
}

#### Affichage des résultats pour des segments de marché spécifiques

Divisons la période en segments : phases haussières, baissières et consolidations. Analysons les performances de l'algorithme sur chaque phase pour évaluer la sensibilité des indicateurs.

In [8]:
// Déterminer les phases du marché
var trendPhases = dailyData.Select((bar, index) => new
{
    Index = index,
    Time = bar.Time,
    Close = bar.Close,
    PreviousClose = index > 0 ? dailyData[index - 1].Close : bar.Close,
    Trend = index > 0
        ? (bar.Close > dailyData[index - 1].Close ? "Bullish" :
           (bar.Close < dailyData[index - 1].Close ? "Bearish" : "Flat"))
        : "Flat"
}).ToList();

// Groupement par tendance
var groupedPhases = trendPhases.GroupBy(p => p.Trend)
    .ToDictionary(g => g.Key, g => g.ToList());

// Affichage des statistiques par tendance
foreach (var trend in groupedPhases)
{
    Console.WriteLine($"Trend: {trend.Key}, Days: {trend.Value.Count}");
    var avgReturn = trend.Value
        .Select(p => (p.Close - p.PreviousClose) / p.PreviousClose)
        .Average();
    Console.WriteLine($"Average Return: {avgReturn:P}");
}


#### Setting Dynamic ADX Percentiles

We introduce parameters `AdxLowerPercentile` and `AdxUpperPercentile` to replicate the logic tested in daily resolution. In the daily approach, we found that (6,86) was good, but hourly might differ. We'll set these as variables here for experimentation.


In [ ]:
int AdxWindowPeriod = 140; // Example from previous optimization
int AdxLowerPercentile = 6; // from daily optimization results, but adjustable
int AdxUpperPercentile = 86;

// Create a rolling window for ADX
var adxWindow = new List<decimal>();


#### Computing ADX for Hourly Data

We compute ADX with the chosen period, then fill a rolling window and compute dynamic percentiles.


In [ ]:
int AdxPeriod = 25; // Example same as daily optimal
var adx = qb.ADX(btcUsdt, AdxPeriod, Resolution.Hour);

foreach (var bar in btcHistoryHour)
{
    adx.Update(bar);
    if (adx.IsReady)
    {
        adxWindow.Add(adx.Current.Value);
    }
}

Console.WriteLine($"ADX Window Count: {adxWindow.Count}");


#### Computing ADX Percentiles

We use a method similar to the daily approach to compute median and percentiles dynamically.


In [ ]:
(decimal median, decimal lowerPercentil, decimal upperPercentil) ComputeAdxPercentiles(List<decimal> values, int lowerPct, int upperPct)
{
    var sorted = values.OrderBy(x => x).ToList();
    int count = sorted.Count;
    if (count == 0) return (0,0,0);

    decimal median = sorted[count/2];

    lowerPct = Math.Min(Math.Max(lowerPct,0),100);
    upperPct = Math.Min(Math.Max(upperPct,0),100);

    int lowerIndex = Math.Max(0, Math.Min(count-1, count * lowerPct / 100));
    int upperIndex = Math.Max(0, Math.Min(count-1, count * upperPct / 100));

    decimal lowerPercentil = sorted[lowerIndex];
    decimal upperPercentil = sorted[upperIndex];

    return (median, lowerPercentil, upperPercentil);
}

var (medianAdx, adxLowVal, adxHighVal) = ComputeAdxPercentiles(adxWindow, AdxLowerPercentile, AdxUpperPercentile);
Console.WriteLine($"Computed ADX percentiles: Lower={adxLowVal:F2}, Upper={adxHighVal:F2}, Median={medianAdx:F2}");


#### Integrating MACD (Hourly)

We choose MACD parameters and compute MACD histogram for hourly data as well.


In [ ]:
int MacdFast = 12;
int MacdSlow = 20;
int MacdSignal = 16;

var macd = qb.MACD(btcUsdt, MacdFast, MacdSlow, MacdSignal, MovingAverageType.Exponential, Resolution.Hour, Field.Close);

var macdHistValues = new List<(DateTime Time, decimal Histogram)>();

foreach (var bar in btcHistoryHour)
{
    macd.Update(bar.EndTime, bar.Close);
    if (macd.IsReady)
    {
        var hist = macd.Current.Value - macd.Signal.Current.Value;
        macdHistValues.Add((bar.EndTime, hist));
    }
}

Console.WriteLine($"MACD ready points: {macdHistValues.Count}");


#### Applying Dynamic Conditions

If we were to simulate signals (not backtesting fully here), we would check if `adxValue >= adxHighVal && isMacdBullish` for buy signals, and `adxValue <= adxLowVal && isMacdBearish` for sell signals.


In [ ]:
// Example pseudo-signal generation
int buySignals = 0;
int sellSignals = 0;

var priceByDateHour = btcHistoryHour.ToDictionary(x => x.EndTime, x => x.Close);

// We'll loop through macdHistValues and find corresponding ADX and check conditions
foreach (var (time, hist) in macdHistValues)
{
    if(!priceByDateHour.ContainsKey(time)) continue;

    var adxVal = adxWindow.Count > 0 ? adxWindow[Math.Min(adxWindow.Count -1, adxWindow.Count -1)] : 0m; 
    // Above line is just a placeholder. In practice, you'd keep track of ADX values aligned with time:
    // For that, store ADX time-value pairs similarly to daily approach.

    var isBullish = hist > 0;
    var isBearish = hist < 0;

    // Suppose we had a dictionary for hourly adx values: 
    // var adxVal = adxHourlyDict.ContainsKey(time) ? adxHourlyDict[time] : 0;

    // Check conditions:
    if (adxVal >= adxHighVal && isBullish) buySignals++;
    else if (adxVal <= adxLowVal && isBearish) sellSignals++;
}

Console.WriteLine($"Buy Signals: {buySignals}, Sell Signals: {sellSignals}");
